In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split

spark = SparkSession.builder.getOrCreate()

In [10]:
# TEST 1 - Stream for socket using nc 
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark\
   .readStream\
   .format('socket')\
   .option('host', 'localhost')\
   .option('port', 9999)\
   .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, ' ')
   ).alias('word')
)

# Generate running word count
wordCounts = words.groupBy('word').count()

# Start running the query that prints the running counts to the console
query = wordCounts\
    .writeStream\
    .outputMode('complete')\
    .format('console')\
    .start()

query.awaitTermination()

KeyboardInterrupt: 

In [ ]:
# TEST 2 - Stream from Kafka
# Subscribe to topic
stream = spark\
    .readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers", "ip-10-0-0-128.us-west-2.compute.internal:9092,ip-10-0-0-213.us-west-2.compute.internal:9092")\
    
    .option("subscribe", "twitter")\
    .load()

# Extract some relevant data about the tweet
# columns = ['created_at',
#            'text',
#            'user.screen_name as user_name']
# tweets_raw = stream.select('created_at','text','user.screen_name')


# tweets = tweets_raw.filter(tweets_raw.created_at.isNotNull())\
#     .withColumn('created_at', unix_timestamp(tweets_raw.created_at, 'EEE MMM dd HH:mm:ss +0000 yyyy'))\
#     .withColumn('created_at', from_unixtime('created_at','YYYY-MM-dd HH:mm:ss'))\
#     .withColumn('created_at', from_utc_timestamp('created_at', 'EST'))

# counts = tweets.groupBy('created_at')\
#     .count()
    
query = stream.selectExpr("CAST(key AS STRING) as key", "CAST(value AS STRING) as value")\
    .groupBy('key')\
    .count()\
    .writeStream\
    .outputMode('complete')\
    .format('console')\
    .start()
    
query.awaitTermination()